In [4]:
from sklearn.datasets import load_breast_cancer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_predict,cross_val_score
from sklearn import model_selection
from sklearn.model_selection import KFold

import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline,make_pipeline

from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import MultinomialNB
import pandas as pd

from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import BaggingClassifier

from sklearn.ensemble import VotingClassifier
from sklearn.feature_extraction.text import CountVectorizer

from sklearn.metrics import precision_score, recall_score, f1_score,classification_report

In [5]:
import warnings
from sklearn.exceptions import DataConversionWarning
warnings.filterwarnings(action='ignore', category=DataConversionWarning)

In [6]:
sms = pd.read_table("sms.tsv",header = None,names = ["label","message"])

In [7]:
sms["label"] = sms.label.map({'ham':0,"spam":1})

In [8]:
X = sms.message
y = sms.label

In [9]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = 1000,random_state = 123)

In [14]:
pipeline_forest = Pipeline([('counter',CountVectorizer(min_df=0.0,max_df=0.8)),
                        ('forest',RandomForestClassifier())])
param_grid = {"forest__n_estimators":[10,50],
              "forest__min_samples_leaf":[5,10,20],
                  "forest__bootstrap":[True,False]}

In [15]:
grid = GridSearchCV(pipeline_forest,
                    param_grid,
                    cv=5,
                    refit=True,
                    n_jobs=3)
grid.fit(X_train,y_train)
    
print(grid.best_params_)
print(grid.best_score_)

{'forest__bootstrap': False, 'forest__min_samples_leaf': 5, 'forest__n_estimators': 50}
0.964348206474


In [29]:
pipeline_bag_tree = Pipeline([('counter',CountVectorizer(min_df=0.0,max_df=0.8)),
                        ('bag_tree',BaggingClassifier(base_estimator=DecisionTreeClassifier()))])
param_grid = {"bag_tree__n_estimators":[10,50],
              "bag_tree__bootstrap":[True,False],
             "bag_tree__max_samples":[0.2,0.5,0.8],
             "bag_tree__base_estimator__min_samples_split":[5,10]}

In [43]:
grid = GridSearchCV(pipeline_bag_tree,
                    param_grid,
                    cv=5,
                    refit=True,
                    n_jobs=3,
                   scoring = 'f1')
grid.fit(X_train,y_train)
    
print(grid.best_params_)
print(grid.best_score_)

{'bag_tree__base_estimator__min_samples_split': 10, 'bag_tree__bootstrap': False, 'bag_tree__max_samples': 0.8, 'bag_tree__n_estimators': 10}
0.892257496547


In [40]:
GridSearchCV?

In [31]:
best_model = grid.best_estimator_

In [34]:
print(classification_report(y_pred=best_model.predict(X_train),y_true=y_train))

             precision    recall  f1-score   support

          0       1.00      1.00      1.00      3965
          1       1.00      1.00      1.00       607

avg / total       1.00      1.00      1.00      4572



In [37]:
607/3965 #niezbalansowane klasy - zamiast accuracy lepiej stosować F1 

0.15308953341740228